In [8]:
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import re

df_column = ["type (actionDetails 0)",
    "url (actionDetails 0)",
    "pageTitle (actionDetails 0)", 
    "pageIdAction (actionDetails 0)" 
        
    ]

def copy_from_stringio(conn, df, person):
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    

# URL to fetch the data
url = "https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    pattern = re.compile(r'\(actionDetails \d+\)')
    filtered_columns = [column for column in df.columns if pattern.search(column)]

   
    if len(filtered_columns) > 1600:
        filtered_columns = filtered_columns[:1600]

    
    df_filtered = df[filtered_columns]
    
  

    # Optional: Print the list of found columns
    # print(f"Filtered columns: {filtered_columns}")

    # Database connection settings
    username = 'postgres'
    password = 'Sharjeel'
    host = 'localhost'
    port = '5432'
    database = 'Pharma'

    # Create the database connection engine
    engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')
    
    # Specify the table name
    table_name = 'person'
    
    # Upload the DataFrame to PostgreSQL
    df_filtered.to_sql(table_name, engine, if_exists='replace', index=False)

    print(f"Data successfully uploaded to the '{table_name}' table in PostgreSQL.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Data successfully uploaded to the 'person' table in PostgreSQL.


In [1]:
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import re
url = "https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1"
response = requests.get(url)
response

<Response [200]>

In [3]:
import requests
import pandas as pd
from io import StringIO

df_column = ["type (actionDetails 0)",
    "url (actionDetails 0)",
    "pageTitle (actionDetails 0)", 
    "pageIdAction (actionDetails 0)" 
        
    ]

# URL to the Matomo API, fetching data in TSV format
url = "https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1"

# Send the GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Read the TSV data into a pandas DataFrame
    data = StringIO(response.text)
    df = pd.read_csv(data, sep='\t')

    # Display the DataFrame's columns
    print("Columns in the data:")
    print(df.columns)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Columns in the data:
Index(['idSite', 'idVisit', 'visitIp', 'visitorId', 'fingerprint',
       'type (actionDetails 0)', 'url (actionDetails 0)',
       'pageTitle (actionDetails 0)', 'pageIdAction (actionDetails 0)',
       'idpageview (actionDetails 0)',
       ...
       'revenueTax (actionDetails 13)', 'revenueShipping (actionDetails 13)',
       'revenueDiscount (actionDetails 13)', 'items (actionDetails 13)',
       'referrerType (actionDetails 13)', 'referrerName (actionDetails 13)',
       'referrerKeyword (actionDetails 13)', 'goalName (actionDetails 5)',
       'goalId (actionDetails 5)', 'goalPageId (actionDetails 5)'],
      dtype='object', length=1796)


In [ ]:
 df_columns = df.columns

    filtered_columns = [col for col in df_columns if col in required_columns]

    df_filtered = df[filtered_columns]

In [ ]:
import requests
import boto3
from datetime import datetime
import pandas as pd
import io

# Define required columns
required_columns = [
    "type (actionDetails 0)",
    "url (actionDetails 0)",
    "pageTitle (actionDetails 0)", 
    "pageIdAction (actionDetails 0)"
]

# Initialize S3 client
s3 = boto3.client('s3')

def lambda_handler(event, context):
    try:
        # Fetch data from the Matomo API
        response = requests.get("https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1")
        matomo_status_code = response.status_code
        
        if matomo_status_code != 200:
            return {
                'statusCode': matomo_status_code,
                'body': f'Failed to retrieve data from Matomo. Status code: {matomo_status_code}'
            }
        
        # Convert TSV to DataFrame
        tsv_data = response.text
        df = pd.read_csv(io.StringIO(tsv_data), sep="\t")
        
        # Filter columns
        filtered_columns = [col for col in df.columns if col in required_columns]
        df_filtered = df[filtered_columns]
        
        # Convert DataFrame to CSV
        csv_buffer = io.StringIO()
        df_filtered.to_csv(csv_buffer, index=False)

        # Get current month for S3 key
        current_month = datetime.now().strftime('%B')
        
        # Define S3 bucket and key
        bucket_name = 'api.s3.bucket'
        s3_key = f'{current_month}/data.csv'
        
        # Upload CSV data to S3
        s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())

        return {
            'statusCode': 200,
            'body': f'Data successfully stored in {bucket_name}/{s3_key}'
        }
    
    except Exception as error:
        return {
            'statusCode': 500,
            'body': f"Error: {str(error)}"
        }
    
    finally:
        print(f"Request completed. Status code: {matomo_status_code if 'response' in locals() else 'No response'}")

